## 加载数据

In [8]:
import numpy as np
from utils import ExtractActivities
from StringIO import StringIO

filename = "./data/ann.txt"
activities = ExtractActivities(filename)
file_bin_sensors = "./data/bin_sensors.txt"
bin_sensors = np.loadtxt(file_bin_sensors,dtype=np.str)
con_sensors = np.loadtxt("./data/continue_sensors.txt",dtype=np.str)
labels = np.loadtxt("./data/activities.txt",dtype=np.str)

#### 清理数据
只剩下二值传感器

In [9]:
def onlybin(activity):
    bin_activity = [activity[0]]
    for i in range(1,len(activity)):
        if len(activity[i]) == 4 and activity[i][2] in bin_sensors:
            bin_activity.append(activity[i])
    return bin_activity

print("开始清理数据，只保留二值传感器...")
activities = ExtractActivities(filename)
# print onlybin(activities[0])
bin_activities = []
# for activity in activities[0:20]:
for activity in activities:
    bin_activities.append(onlybin(activity))
print bin_activities[0]
bin_activities = np.array(bin_activities)
activity_labels = [activity[0] for activity in bin_activities]
print bin_activities.shape
print("清理结束...")

开始清理数据，只保留二值传感器...
['Sleep', ['2011-06-15', '02:13:20.812509', 'M007', 'ON'], ['2011-06-15', '02:13:21.919635', 'M007', 'OFF'], ['2011-06-15', '05:40:14.111389', 'M021', 'ON']]
(2655L,)
清理结束...


计算TF

In [10]:
import pandas as pd
# print bin_sensors
df_activity_sensor = pd.DataFrame(index=np.arange(bin_activities.shape[0]),columns=bin_sensors)
activity_sensor_times = np.array([[0 for x in range(len(bin_sensors))] 
                                  for i in range(len(bin_activities))],dtype=np.float16)

for i in range(0,len(bin_activities)):
    current_act = bin_activities[i]
    for j in range(1,len(current_act)):
        col_index = list(bin_sensors).index(current_act[j][2])
        activity_sensor_times[i,col_index] += 1
total_times = activity_sensor_times.sum(axis=1)
tf = activity_sensor_times / total_times[:,np.newaxis]
print tf.shape
print("end")

(2655L, 37L)
end


计算IDF

In [11]:
ac_one = np.zeros((len(bin_activities),len(bin_sensors)))
for i in range(ac_one.shape[0]):
    for j in range(ac_one.shape[1]):
        if activity_sensor_times[i,j] > 0:
            ac_one[i,j] = 1
is_has_sensor = ac_one.sum(axis=1)  
# 文档频率
df = float(ac_one.shape[0])/is_has_sensor
idf = np.log((df+0.01))
print idf.shape

print("end")

(2655L,)
end


### 计算tfidf

In [12]:
sensor_weight = tf * idf[:,np.newaxis]

df_sensor_weights = pd.DataFrame(sensor_weight,columns=bin_sensors,
                                 index=range(len(bin_activities)))
df_sensor_weights.insert(0,'label',activity_labels)
df_sensor_weights[df_sensor_weights['label']=='Sleep'].describe()
print("end")

end


### 加入Entry

In [13]:
tfidfentrys = list()
targets = []
activity_labels = np.array(activity_labels)

for label in labels:
    current_activity_index = (activity_labels == label)
    # 该类行为的传感器次数矩阵，包含所有的传感器
    current_activity_sensor_times = activity_sensor_times[current_activity_index]

    # 该类行为出现的总次数
    current_activity_times = sum(current_activity_index)

    # 该类行为的传感器权重
    current_activity_sensor_weight = sensor_weight[current_activity_index]

    # 得到当前类中出现的所有传感器下标
    index = current_activity_sensor_weight.sum(axis=0)>0

    # 获取当前行为触发的所有传感器ID
    current_activity_actsensor = bin_sensors[index]
    # print current_activity_actsensor

    # 该类行为触发的所有传感器次数，只包含触发的传感器
    current_activity_actsensor_times = current_activity_sensor_times[:,index]
    current_activity_actsensor_weight = current_activity_sensor_weight[:,index]

    # 该类每种触发的传感器在所有属于该类的样本中出现的总次数
    num = np.zeros((1,len(current_activity_actsensor_times[1])))
    total_times_sensors = current_activity_actsensor_times.sum(axis=0)
    fenmu = total_times_sensors + 0.01
    fenzi = current_activity_actsensor_times +0.01

    # 该类行为激发的传感器的熵Etc：传感器t在c类内的分布熵
    entry = sum(-(fenzi/fenmu) * np.log(fenzi/fenmu))
    
    entry_all = np.zeros(len(bin_sensors))
    entry_all[index] = entry
    # print entry_all
    tfidfe = current_activity_sensor_weight * entry_all
#     print("tfidfe.shape",tfidfe.shape)
    tfidfentrys.extend(tfidfe)
#     print("tfidfentrys",np.shape(tfidfentrys))
    targets.append([label] * current_activity_times )
    
# sensor_entry_inner_class = 

# 还原成最初的shape
print(np.shape(tfidfentrys))
print("end")

(2655L, 37L)
end


找到对每类行为最重要的若干个传感器

In [35]:
tfidfentrys = np.array(tfidfentrys)
for label in labels:
    current_activity_index = (activity_labels == label)
    current_activity_tfidfe = tfidfentrys[current_activity_index]
    print current_activity_tfidfe.shape
    total_tfidfe_sensors = current_activity_tfidfe.sum(axis=0) > 0
#     sensor_index = (total_tfidfe_sensors)
#     print bin_sensors[total_tfidfe_sensors]

(44L, 37L)
(49L, 37L)
(33L, 37L)
(48L, 37L)
(79L, 37L)
(62L, 37L)
(119L, 37L)
(36L, 37L)
(52L, 37L)
(42L, 37L)
(49L, 37L)
(147L, 37L)
(16L, 37L)
(14L, 37L)
(62L, 37L)
(234L, 37L)
(32L, 37L)
(462L, 37L)
(55L, 37L)
(82L, 37L)
(19L, 37L)
(106L, 37L)
(7L, 37L)
(60L, 37L)
(173L, 37L)
(43L, 37L)
(30L, 37L)
(30L, 37L)
(39L, 37L)
(129L, 37L)
(7L, 37L)
(37L, 37L)
(258L, 37L)


### 加入IG